## **Objetivo:**
### Prever se um **Cliente** irá realizar uma compra nos próximos **30 dias**
---

## **Importando e Tranformando os dados**

In [0]:
!pip install xgboost

In [0]:
%restart_python

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pyspark.sql.functions as F

In [0]:
tabela_bronze = "estudo.default.tvendas_bronze"
tabela_cluster_gold = "estudo.default.tcluster_cli_gold"
tabela_modelo_previsao_compra = "estudo.default.tprev_compra_silver"

In [0]:
df_raw_spark = spark.table(tabela_bronze)
df_raw = df_raw_spark.toPandas()

df_clientes_spark = spark.table(tabela_cluster_gold).filter(F.col("ds_cluster") == 'Cliente Comum').select("fk_contact", "ds_cluster")
df_clientes = df_clientes_spark.toPandas()

# Verificando os tipos das colunas
df_raw.info()
df_clientes.info()

### 🎯 **Segmento Foco: Clientes Comuns**

Optamos por focar o modelo nos **Clientes Comuns**, pois são o grupo com **histórico de compras consistente**, **features bem definidas** e **alto potencial de previsibilidade**. Mesmo sendo uma porcentagem pequena, representam o maior retorno possível com campanhas bem direcionadas.


In [0]:
# Pegando somente os clientes do segmento "Cliente Comum"
clientes_comuns = df_clientes[df_clientes['ds_cluster'] == 'Cliente Comum']['fk_contact'].reset_index(drop=True).to_frame()

print(f"Total de Clientes Comuns: {len(clientes_comuns)}")
clientes_comuns.head()

## **Tratando os dados**

In [0]:
# Passando as colunas para o tipo date
df_raw['date_purchase'] = pd.to_datetime(df_raw['date_purchase'])
df_raw['time_purchase'] = pd.to_datetime(df_raw['time_purchase'], format='%H:%M:%S').dt.time

In [0]:

df_raw['gmv_success'] = df_raw['gmv_success'].astype(float)

valores_negativo_ou_zero = df_raw[df_raw['gmv_success'] <= 0]

print(f'Existem {valores_negativo_ou_zero["gmv_success"].count()} valores menores ou igual a zero')

# Excluindo as linhas com valores negativos
df_raw = df_raw[~df_raw['nk_ota_localizer_id'].isin(valores_negativo_ou_zero['nk_ota_localizer_id'])]

---


##  Divisão dos Dados: **Clientes de Treino** & **Clientes de Teste**

> **80%** dos clientes serão usados para **treinar** o modelo  

> **20%** serão guardados para **testar** o modelo

In [0]:
from sklearn.model_selection import train_test_split

# Definindo uma semente aleatória
SEED = 358477

# A função train_test_split separa os clientes em treino e teste automaticamente
clientes_treino, clientes_teste = train_test_split(
    clientes_comuns['fk_contact'],
    test_size=0.2,
    random_state=SEED
)

print(f"Número de clientes separados para treino: {len(clientes_treino)}")
print(f"Número de clientes separados para teste: {len(clientes_teste)}")

In [0]:
# Estamos filtrando o dataframe de compras para possuir apenas os clientes comuns
df_compras_treino = df_raw[df_raw['fk_contact'].isin(clientes_treino)]
df_compras_teste = df_raw[df_raw['fk_contact'].isin(clientes_teste)]

print(f"\nNúmero de compras no conjunto de treino: {len(df_compras_treino)}")
print(f"Número de compras no conjunto de teste: {len(df_compras_teste)}")

# **Feature Engeneering**

## Criando Features para o Conjunto de **Treino**

In [0]:
ultima_data_treino = df_compras_treino['date_purchase'].max()
data_corte_treino = ultima_data_treino - pd.DateOffset(days=30)

print(f"A data de corte para o treino será: {data_corte_treino.date()}")

# Filtrando as compras até a data de corte
features_treino_df = df_compras_treino[df_compras_treino['date_purchase'] <= data_corte_treino].copy()

##### Não podemos pegar as métricas RFM do df_cluster pois estaríamos influenciando os dados, causando um vazemento de dados entre os clientes de treino e teste

In [0]:
# --- Recência, Frequência e Valor Gasto ---
df_final_treino = features_treino_df.groupby('fk_contact').agg(
    recencia=('date_purchase', lambda x: (data_corte_treino - x.max()).days),
    frequencia=('date_purchase', 'count'),
    valor_total_gasto=('gmv_success', 'sum')
).reset_index()

# --- Média de Tempo entre Compras ---
df_intervalos = features_treino_df.sort_values(by=['fk_contact', 'date_purchase'])
df_intervalos['intervalo_dias'] = df_intervalos.groupby('fk_contact')['date_purchase'].diff().dt.days

df_media_tempo = df_intervalos.groupby('fk_contact')['intervalo_dias'].mean().reset_index()
df_media_tempo.columns = ['fk_contact', 'media_tempo_entre_compras']

df_final_treino = pd.merge(df_final_treino, df_media_tempo, on='fk_contact', how='left')

# --- Numero de Rotas Únicas ---
features_treino_df['rota'] = features_treino_df['place_origin_departure'] + ' -> ' + features_treino_df['place_destination_departure']
df_rotas_unicas = features_treino_df.groupby('fk_contact')['rota'].nunique().reset_index()
df_rotas_unicas.columns = ['fk_contact', 'num_rotas_unicas']

df_final_treino = pd.merge(df_final_treino, df_rotas_unicas, on='fk_contact', how='left')

# --- Número de Viagens com Retorno ---
df_com_retorno = features_treino_df[features_treino_df['place_destination_return'] != '0'].groupby('fk_contact').size().reset_index(name='num_viagens_com_retorno')

df_final_treino = pd.merge(df_final_treino, df_com_retorno, on='fk_contact', how='left')

# --- Média de Passagens por Viagem ---
df_media_passagens = features_treino_df.groupby('fk_contact')['total_tickets_quantity_success'].mean().reset_index()
df_media_passagens.columns = ['fk_contact', 'media_passagens_por_viagem']

df_final_treino = pd.merge(df_final_treino, df_media_passagens, on='fk_contact', how='left')

# Dias desde a primeira compra (tempo de vida do cliente)
df_primeira_compra = features_treino_df.groupby('fk_contact')['date_purchase'].min().reset_index()
df_primeira_compra.columns = ['fk_contact', 'primeira_compra']
df_primeira_compra['dias_desde_primeira_compra'] = (data_corte_treino - df_primeira_compra['primeira_compra']).dt.days

df_final_treino = pd.merge(df_final_treino, df_primeira_compra[['fk_contact', 'dias_desde_primeira_compra']], on='fk_contact', how='left')  


# --- Tratamento de Nulos ---
df_final_treino.fillna(0, inplace=True)

df_final_treino.head()

### **Preparação do x_treino**

In [0]:
# Garantimos que o x_treino contenha exatamente todos os clientes separados para o treino
x_treino = pd.DataFrame(clientes_treino, columns=['fk_contact'])
x_treino = pd.merge(x_treino, df_final_treino, on='fk_contact', how='left')

x_treino.head(3)

### **Criando a Variável Alvo**

In [0]:
alvo_df_treino = df_compras_treino[df_compras_treino['date_purchase'] > data_corte_treino]

clientes_compradores_treino = alvo_df_treino['fk_contact'].unique()

x_treino['alvo'] = x_treino['fk_contact'].isin(clientes_compradores_treino).astype(int)
x_treino.head(3)

### **Filtrando as colunas para o x_treino e y_treino**

In [0]:
y_treino = x_treino['alvo']
x_treino = x_treino.drop(columns=['fk_contact', 'alvo'])

---

## Criando Features para o Conjunto de **Teste**

#### Iremos aplicar as mesmas etapas, porém no conjunto de teste

In [0]:
ultima_data_teste = df_compras_teste['date_purchase'].max()
data_corte_teste = ultima_data_teste - pd.DateOffset(days=30)

features_teste_df = df_compras_teste[df_compras_teste['date_purchase'] <= data_corte_teste].copy()

print(f"A data de corte para o teste será: {data_corte_teste.date()}")

In [0]:
# --- Recência, Frequência e Valor Gasto ---
df_final_teste = features_teste_df.groupby('fk_contact').agg(
    recencia=('date_purchase', lambda x: (data_corte_teste - x.max()).days),
    frequencia=('date_purchase', 'count'),
    valor_total_gasto=('gmv_success', 'sum')
).reset_index()

# --- Média de Tempo entre Compras ---
df_intervalos_teste = features_teste_df.sort_values(by=['fk_contact', 'date_purchase'])
df_intervalos_teste['intervalo_dias'] = df_intervalos_teste.groupby('fk_contact')['date_purchase'].diff().dt.days
df_media_tempo_teste = df_intervalos_teste.groupby('fk_contact')['intervalo_dias'].mean().reset_index()
df_media_tempo_teste.columns = ['fk_contact', 'media_tempo_entre_compras']

df_final_teste = pd.merge(df_final_teste, df_media_tempo_teste, on='fk_contact', how='left')

# --- Numero de Rotas Únicas ---
features_teste_df['rota'] = features_teste_df['place_origin_departure'] + ' -> ' + features_teste_df['place_destination_departure']
df_rotas_unicas_teste = features_teste_df.groupby('fk_contact')['rota'].nunique().reset_index()
df_rotas_unicas_teste.columns = ['fk_contact', 'num_rotas_unicas']

df_final_teste = pd.merge(df_final_teste, df_rotas_unicas_teste, on='fk_contact', how='left')

# --- Número de Viagens com Retorno ---
df_com_retorno_teste = features_teste_df[features_teste_df['place_destination_return'] != '0'].groupby('fk_contact').size().reset_index(name='num_viagens_com_retorno')

df_final_teste = pd.merge(df_final_teste, df_com_retorno_teste, on='fk_contact', how='left')

# --- Média de Passagens por Viagem ---
df_media_passagens_teste = features_teste_df.groupby('fk_contact')['total_tickets_quantity_success'].mean().reset_index()
df_media_passagens_teste.columns = ['fk_contact', 'media_passagens_por_viagem']

df_final_teste = pd.merge(df_final_teste, df_media_passagens_teste, on='fk_contact', how='left')

# Dias desde a primeira compra (tempo de vida do cliente)
df_primeira_compra = features_teste_df.groupby('fk_contact')['date_purchase'].min().reset_index()
df_primeira_compra.columns = ['fk_contact', 'primeira_compra']
df_primeira_compra['dias_desde_primeira_compra'] = (data_corte_teste - df_primeira_compra['primeira_compra']).dt.days
df_final_teste = pd.merge(df_final_teste, df_primeira_compra[['fk_contact', 'dias_desde_primeira_compra']], on='fk_contact', how='left')  


# --- Tratamento de Nulos ---
df_final_teste.fillna(0, inplace=True)

df_final_teste.head()

### **Preparação do x_teste**

In [0]:
x_teste = pd.DataFrame(clientes_teste.values, columns=['fk_contact'])
x_teste = pd.merge(x_teste, df_final_teste, on='fk_contact', how='left')

### **Criando a Variável Alvo**

In [0]:
alvo_teste_df = df_compras_teste[df_compras_teste['date_purchase'] > data_corte_teste]

clientes_compradores_teste = alvo_teste_df['fk_contact'].unique()
x_teste['alvo'] = x_teste['fk_contact'].isin(clientes_compradores_teste).astype(int)

### **Filtrando as colunas para o x_teste e y_teste**

In [0]:
y_teste = x_teste['alvo']
x_teste = x_teste.drop(columns=['fk_contact', 'alvo'])

---

### **Verificando a distribuição de compradores e não compradores**

In [0]:
y_treino.value_counts(normalize=True)

In [0]:
y_teste.value_counts(normalize=True)

#### A variável alvo segue segue a mesma proporção tanto no **y_treino** como no **y_teste**, isso é muito bom para nosso modelo

## **Dummy Classifier**
---

Iremos começar com o **Dummy Classifier** para ter nossa linha de base de **precisão** e **recall**. O modelo irá “chutar” que todos os clientes **não** vão fazer uma compra no mês seguinte, pois essa é a variável mais frequente.  

Como, em média, **70%** dos clientes não compraram, a **acurácia será de 70%**.  

Porém, queremos ver os acertos quando se trata de quem **realmente comprou**. Nesse caso, a **precisão é de apenas 27%** e o **recall de 30%**, resultando em um **F1-score de 28%**, o que não é nada bom.


In [0]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import classification_report

# A estratégia 'stratified' faz previsões aleatóris, mas mantendo a mesma distribuição de classes do conjunto de treino.
dummy_clf = DummyClassifier(strategy="stratified", random_state=SEED)

# Treinando o modelo com os mesmos dados de treino
dummy_clf.fit(x_treino, y_treino)

# Fazendo previsões no conjunto de teste
y_pred_dummy = dummy_clf.predict(x_teste)

# Exibindo o relatório de classificação
print("Relatório de Classificação (Dummy Classifier):")
print(classification_report(y_teste, y_pred_dummy, target_names=['Não Comprou (0)', 'Comprou (1)']))

# **XGBoost**

**XGBoost** é um algoritmo poderoso de *gradient boosting* que oferece alta precisão e rapidez, mesmo em problemas com classes desbalanceadas.

Com ajustes de pesos para as classes, ele melhora a predição da classe minoritária, sendo ideal para identificar **não compradores**.


In [0]:
import xgboost as xgb
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Calculando o peso da classe positiva (quem comprou). Isso ajuda o XGBoost a dar mais atenção pra essa classe minoritária.
peso = y_treino.value_counts()[0] / y_treino.value_counts()[1]
print(f"Peso calculado para o treino final: {peso:.2f}")


# Montamos o pipeline final que vai escalar os dados
pipeline_xgb_final = Pipeline([
    ('scaler', StandardScaler()),    # Normaliza os dados pra facilitar o treino
    ('xgb', xgb.XGBClassifier(
        learning_rate=0.05,          # Taxa de aprendizado 
        max_depth=7,                 # Profundidade das árvores (complexidade do modelo)
        n_estimators=600,            # Quantidade de árvores que o modelo vai criar
        colsample_bytree=0.8,        # Amostra de colunas usadas em cada árvore, ajuda a evitar overfitting
        objective='binary:logistic', # Problema é de classificação binária
        scale_pos_weight=peso,       # Ajusta o peso da classe positiva pra lidar com desbalanceamento
        eval_metric='logloss',       # Métrica usada pra avaliar a perda durante o treino
        random_state=SEED            # Pra deixar os resultados reproduzíveis
    ))
])

# Treinando o modelo XGBoost final com os dados de treino
pipeline_xgb_final.fit(x_treino, y_treino) 

# Fazendo a previsão com os dados de teste
y_pred_final = pipeline_xgb_final.predict(x_teste)  

# Calcula a acurácia geral do modelo, mesmo não sendo a melhor métrica para avaliar o modelo
accuracy_final = accuracy_score(y_teste, y_pred_final)
print(f"Acurácia  do Modelo: {accuracy_final:.2%}")

# Mostra o relatório completo de classificação, mostrando precisão, recall e F1 por classe
print("\nRelatório de Classificação:")
print(classification_report(y_teste, y_pred_final, target_names=['Não Comprou (0)', 'Comprou (1)']))


## **Trade-Off: Precisão x Recall**

##### Nosso modelo obteve uma média de **65%** entre a precisão e o recall, porém, na nossa estratégia, iremos focar em aumentar a **recall**. Queremos uma 'certeza' de pelo menos **30%**, ou seja, o modelo só vai dizer que o cliente vai comprar se tiver mais de **30% de certeza**.  
 
##### Ao diminuir o limite de decisão, aumentamos o recall, mas corremos o risco de reduzir a precisão, já que o modelo passará a classificar mais casos como positivos — inclusive alguns que não comprarão. É uma **troca (trade-off)**


In [0]:
# Pegar as probabilidades do modelo para o conjunto de teste 
y_probabilities_final = pipeline_xgb_final.predict_proba(x_teste)[:, 1]

# 2. Definir o nossa linha de trade-off. Vamos usar um limiar de 0.3, que é um pouco mais baixo que o padrão de 0.5.
nosso_limiar_ideal = 0.3

# Aplicar essa regra para transformar as probabilidades em previsão final (0 ou 1)
y_pred_com_limiar_ajustado = (y_probabilities_final >= nosso_limiar_ideal).astype(int)

# Mostrar o resultado

print(classification_report(y_teste, y_pred_com_limiar_ajustado, target_names=['Não Comprou (0)', 'Comprou (1)']))


## **Analisando a Matriz de Confusao**

- ✅ Verdadeiro Negativo: O modelo **acertou** que 771 clientes **não iriam comprar**
- ✅ Verdadeiro Positivo: O modelo **acertou** que 344 clientes **iriam comprar**

- ❌ Falso Positivo: O modelo previu que 292 clientes **iriam comprar**, **mas se enganou**
- ❌ Falso Negativo: O modelo previu que 91 clientes **não iriam comprar**, **mas eles compram**

##### Ou seja, dos **435** clientes que **COMPRAM**, o modelo conseguiu prever corretamente 344 clientes (**79,58%**). Na contramão, o modelo afirmou **292** clientes comprariam, mas errou. Obtivemos um recall ótimo, mas a precisão caiu para **54,10%**, pensando na estratégia de não deixar escapar possíveis compradores, já que como analisamos na segmentação de clientes, a ClickBus possui um grande problema na retenção de novos clientes, então esse é um ótimo resultado.

In [0]:
matriz_de_confusao = confusion_matrix(y_teste, y_pred_com_limiar_ajustado)

# Plotando a matriz usando um mapa de calor (heatmap) para melhor visualização
plt.figure(figsize=(8, 6))
sns.heatmap(matriz_de_confusao, 
            annot=True,      # Mostrando os números dentro de cada quadrado
            fmt='d',         # Formatando os números como inteiros
            cmap='Blues',    
            xticklabels=['Previsto: Não Comprou', 'Previsto: Comprou'],
            yticklabels=['Real: Não Comprou', 'Real: Comprou'])

plt.xlabel('Previsão do Modelo', fontsize=12)
plt.ylabel('Valor Real', fontsize=12)
plt.title('Matriz de Confusão do Modelo Final', fontsize=14)
plt.show()


### **Avaliação do Modelo com Curva ROC**

##### A **curva de ROC** ajuda a entender como um modelo se comporta em termos de acerto e erro ao variar o **limiar de decisã**o. Uma **AUC** mais alta indica que o modelo tem maior **capacidade de distinguir entre as duas classes**. No nosso caso, obtivemos um ótimo resultado, um AUC de **85,37%**

In [0]:
from sklearn.metrics import roc_curve, roc_auc_score

# Pegamos as probabilidades que o modelo deu para cada cliente ser comprador
y_probabilities_final = pipeline_xgb_final.predict_proba(x_teste)[:, 1]

# Calculamos a área sob a curva ROC, que mostra o quão bom o modelo é em geral
auc_score = roc_auc_score(y_teste, y_probabilities_final)
print(f"A Área Sob a Curva (AUC) do nosso modelo é: {auc_score:.4f}")

# Calculamos os valores que precisamos para desenhar a curva ROC:
# a taxa de falsos positivos, a taxa de verdadeiros positivos, e os limiares usados
fpr, tpr, thresholds = roc_curve(y_teste, y_probabilities_final)

# Começamos a plotar a curva, mostrando como o modelo se comporta
plt.figure(figsize=(10, 7))
plt.plot(fpr, tpr, marker='.', label=f'XGBoost (AUC = {auc_score:.2f})')

# Adicionamos a linha do “chute aleatório”, que serve como referência para ver se nosso modelo é melhor que um palpite
plt.plot([0, 1], [0, 1], linestyle='--', color='red', label='Chute Aleatório (AUC = 0.5)')

# Colocamos os títulos e legendas
plt.xlabel('Taxa de Falsos Positivos (FPR)')
plt.ylabel('Taxa de Verdadeiros Positivos (TPR / Recall)')
plt.title('Curva ROC')
plt.legend()
plt.grid(True)
plt.show()


### **Análise do Modelo: Padrão Identificado na Feature de Recência**

##### O modelo identificou que a **recência** é uma variável-chave para prever o comportamento de compra dos clientes. Clientes que realizaram compras **recentemente** têm uma **alta probabilidade** de comprar novamente no mês seguinte, enquanto aqueles que **não compram há mais tempo** tendem a apresentar **menor propensão** a novas compras.

##### Esse insight é **fundamental** para direcionar estratégias de **marketing e retenção**, permitindo focar esforços nos clientes com maior potencial de recompra.


In [0]:
from matplotlib import font_manager as fm

# Fazer as previsões com o modelo XGBoost já treinado usando os dados de teste
y_pred_xgboost = pipeline_xgb_final.predict(x_teste)

# Criar um DataFrame juntando as variáveis de teste com as respostas reais e as previsões do modelo
df_resultados_xgboost = x_teste.copy()
df_resultados_xgboost['alvo_real'] = y_teste
df_resultados_xgboost['previsao'] = y_pred_com_limiar_ajustado

# Função para classificar cada previsão em verdadeiro positivo, verdadeiro negativo, falso positivo ou falso negativo
def classificar_resultado(row):
    if row['alvo_real'] == 1 and row['previsao'] == 1:
        return 'Verdadeiro Positivo — Acertou que ia comprar'
    elif row['alvo_real'] == 0 and row['previsao'] == 0:
        return 'Verdadeiro Negativo — Acertou que não ia comprar,'
    elif row['alvo_real'] == 0 and row['previsao'] == 1:
        return 'Falso Positivo — Previu compra, mas não comprou'
    else:  # alvo_real == 1 and previsao == 0
        return 'Falso Negativo — Previu que não compraria, mas comprou'

# Aplicar essa classificação para cada linha do DataFrame
df_resultados_xgboost['resultado'] = df_resultados_xgboost.apply(classificar_resultado, axis=1)

# Calcular quantos acertos e erros o modelo teve
acertos = df_resultados_xgboost['resultado'].str.startswith('Verdadeiro').sum()
erros = df_resultados_xgboost['resultado'].str.startswith('Falso').sum()

print(f"Total de acertos: {acertos}")
print(f"Total de erros: {erros}")
print(f"Taxa de acerto geral: {acertos / (acertos + erros) * 100:.2f}%")

# Criar um gráfico de dispersão para visualizar como os acertos e erros estão distribuídos
plt.figure(figsize=(15, 10))
sns.scatterplot(
    data=df_resultados_xgboost,
    x='recencia',
    y='frequencia',
    hue='resultado',
    palette = {
        'Verdadeiro Positivo — Acertou que ia comprar': '#2ecc71', 
        'Verdadeiro Negativo — Acertou que não ia comprar,': '#3498db', 
        'Falso Positivo — Previu compra, mas não comprou': '#f39c12', 
        'Falso Negativo — Previu que não compraria, mas comprou': '#e74c3c' 
    },
    hue_order=[
        'Verdadeiro Positivo — Acertou que ia comprar',
        'Verdadeiro Negativo — Acertou que não ia comprar,',
        'Falso Positivo — Previu compra, mas não comprou',
        'Falso Negativo — Previu que não compraria, mas comprou'
    ],
    alpha=0.7,
    s=80
)

# Definir títulos e labels
plt.title('Análise Visual dos Acertos e Erros (XGBoost)', fontdict={'fontsize': 16}, pad=15)
plt.xlabel('Recência (Dias desde a última compra)')
plt.ylabel('Frequência (Total de compras)')

# Ajustar a legenda para ficar com título em negrito e fonte maior
plt.legend(title='Resultado da Previsão', title_fontproperties=fm.FontProperties(weight='bold', size=12), fontsize=11)

plt.grid(True)
plt.show()

#### Podemos confirmar a importância da **recência** analisando o atributo `.feature_importances_` do modelo XGBoost, que indica o peso de cada feature na decisão do modelo. Em seguida, plotamos um gráfico para visualizar essas importâncias de forma clara.


In [0]:
# Acessar o modelo XGBoost dentro do pipeline já treinado
modelo_final_acessado = pipeline_xgb_final.named_steps['xgb']

# Pegar a importância que o modelo deu para cada feature
importancias = modelo_final_acessado.feature_importances_

# Montar um DataFrame para organizar e ordenar as importâncias
df_importancias_final = pd.DataFrame({
    'feature': x_treino.columns,
    'importance': importancias
}).sort_values(by='importance', ascending=False)

# Plotar um gráfico de barras para visualizar quais features são mais relevantes
plt.figure(figsize=(10, 6))
plt.figure(figsize=(10, 6))
sns.barplot(
    x='importance',
    y='feature',
    data=df_importancias_final,
    palette='rocket'
)
plt.title('Importância das Features - Clientes Comuns')
plt.xlabel('Score de Importância (XGBoost)')
plt.ylabel('Feature')
plt.show()


# **Entrega Final dos Dados**

In [0]:
# Pegando os IDs dos clientes correspondentes ao x_teste
ids_para_entrega = df_final_teste.loc[x_teste.index, 'fk_contact']

# Criando o DataFrame final de entrega
df_entrega = pd.DataFrame({
    'fk_contact': ids_para_entrega,
    'alvo_real': y_teste,  
    'previsao': y_pred_com_limiar_ajustado,
    'probabilidade_de_compra': y_probabilities_final
})

# Criando uma coluna que verifica se o modelo acertou ou errou
df_entrega['resultado'] = np.where(df_entrega['alvo_real'] == df_entrega['previsao'], 'Acerto', 'Erro')

# Ajustando o índice
df_entrega = df_entrega.set_index(x_teste.index)

df_entrega.head()

### Salvando o DataFrame final em um arquivo CSV
# df_entrega.to_csv('entrega_final.csv', index=False)

---

## **Conclusão do Projeto**

### **Objetivo**
Criar um modelo para prever se clientes "Comuns" vão comprar de novo nos próximos 30 dias.

### Resultados
- **AUC:** 84% — separa muito bem quem compra e quem não compra.  
- **Threshold:** 0,3 para priorizar **Recall**.  
- **Recall:** 79% (344 de 435 compradores identificados).  
- **Precisão:** 54% (trade-off esperado por priorizar recall).  
- **F1-Score:** Evoluímos de 28% para 64%.  
- **Acurácia:** 74,4% no total.  

### Insights
- **Recência importa:** clientes que compraram recentemente têm maior chance de voltar a comprar.  
- **Engajamento pesa:** frequência de compras e gasto total são fatores relevantes.  
- **Padrão claro:** baixa recência + alta frequência = alta chance de recompra.  

### Próximos Passos
1. Criar campanhas direcionadas a clientes com probabilidade de compra superior a 30%.  
2. Focar em ações de reativação para clientes que estão há mais tempo sem comprar.  
3. Rodar previsões semanais/mensais e ajustar o modelo conforme os resultados.  
